<a href="https://colab.research.google.com/github/8Raouf24/PFE/blob/master/Filtrage_Sem_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fse

In [ ]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,callbacks
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, GRUCell, GRU, Dropout
from keras.models import Sequential
from pathlib import Path
from sklearn.metrics import r2_score
import re
import gzip
import matplotlib.pyplot as plt
import json
import math

In [ ]:
import seaborn as sns
import spacy
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
from string import punctuation
import collections
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import cross_val_score

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Software.json.gz

--2020-10-02 21:14:43--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFiles/Software.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 98221392 (94M) [application/octet-stream]
Saving to: ‘Software.json.gz.2’

Software.json.gz.2  100%[===================>]  93.67M  18.7MB/s    in 6.1s    

2020-10-02 21:14:50 (15.4 MB/s) - ‘Software.json.gz.2’ saved [98221392/98221392]



In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Software.json.gz


--2020-10-02 21:14:50--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Software.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15033606 (14M) [application/octet-stream]
Saving to: ‘meta_Software.json.gz.2’

meta_Software.json. 100%[===================>]  14.34M  8.48MB/s    in 1.7s    

2020-10-02 21:14:52 (8.48 MB/s) - ‘meta_Software.json.gz.2’ saved [15033606/15033606]



In [ ]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        d = json.loads(l)
        yield d
      
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
reviews = getDF('/content/Software.json.gz')

In [ ]:
len(reviews)

459436

In [ ]:
metadata =getDF('/content/meta_Software.json.gz')

Prétraitement description


In [ ]:
metadata['description'].replace('', np.nan, inplace=True)
df_data = metadata.dropna()
df_data.reset_index(drop=True, inplace=True)


In [ ]:
df_data = df_data[['asin','description','title','price','image']]

In [ ]:
df_data['description'] = df_data['description'].apply(lambda x: ','.join(map(str, x )))

In [ ]:
df_data['description'].drop_duplicates(inplace=True)

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nlp = spacy.load("en_core_web_sm")
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def furnished(text):
    final_text = []
    for i in text.split():
        if i.lower() not in stop:
            word = lemmatizer.lemmatize(i)
            final_text.append(word.lower())
    return " ".join(final_text)

df_data.description = df_data.description.apply(remove_html)
df_data.description = df_data.description.apply(remove_punctuation)            
df_data.description = df_data.description.apply(furnished)

In [ ]:
df_data['description'].drop_duplicates(inplace=True)

In [ ]:
df_data['description_training'] = [i.lower().replace(',', '').split() for i in df_data['description']]

In [ ]:
from gensim.models import FastText
ft = FastText(df_data['description_training'], min_count=1, size=50)

In [ ]:
from fse.models import Average
from fse import IndexedList
model = Average(ft)
model.train(IndexedList(list(df_data['description_training'])))



(22516, 3146070)

In [ ]:
model.sv.vectors[11]

array([ 0.62046766,  0.8916182 , -0.2029101 ,  1.1848599 ,  0.23217614,
       -0.44796148,  0.22203985,  0.18531142, -0.25368935, -0.66326535,
       -0.26585743,  0.15896799, -0.6016706 , -1.2442504 ,  0.3012646 ,
       -0.00804526, -0.6862992 , -0.14232706, -0.5469989 , -1.0787435 ,
       -0.11603503,  0.5713417 , -0.15305674,  0.15944079, -0.6624337 ,
       -0.35722753,  0.6458592 ,  0.9310529 , -1.4117945 ,  0.04749518,
       -0.4903742 ,  1.1441587 ,  0.5698838 , -0.32505688,  1.1164767 ,
       -0.5447184 , -0.6704086 ,  0.20247065, -0.31192294,  0.63108814,
        0.18182585, -0.57360315, -0.3982226 ,  0.6256388 ,  0.39240372,
        0.46110877,  0.01484032,  0.55483985, -0.4254316 ,  0.10690609],
      dtype=float32)

In [ ]:
len(model.sv.vectors)

25938

In [ ]:
def listToString(s):  
    # initialize an empty string 
    str1 = " " 
    # return string   
    return (str1.join(s)) 

In [ ]:
df_data['description_training'].apply(lambda x: listToString(x))

0        whats included full activation key full downlo...
1                                                         
2        translation express kingsoft product designed ...
3        powerword designed people learning practicing ...
4        item come cd sleeve ox manual hence price item...
                               ...                        
25933    features 1 beautiful fabulously detailed real ...
25934    features 1 beautiful fabulously detailed real ...
25935    mac internet security x9 contains two best sel...
25936    versacheck x9 quickbooks 2016 dna secure 100 c...
25937    thank interest volume one warkel graphics cnc ...
Name: description_training, Length: 25938, dtype: object

In [ ]:
def getIndexes(dfObj, value):
    ''' Get index positions of value in dataframe i.e. dfObj.'''
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos[0]

In [ ]:
def recommandation(asin):
  idx = df_data[df_data['asin'] == asin].index
  list_rec_idx = []
  
  top_5 = model.sv.most_similar(idx, indexable=sentences)[1:6]
  
  for i in top_5:
    print(df_data['title'][i[1]])
 



  
  


In [ ]:
recommandation('9814272183')

NameError: ignored

Prétraitement du prix

In [ ]:
#On enleve le signe dollars 
df['price']=df['price'].str.slice(1,-1)

In [ ]:
#A l'aide d'une regex , nous prenons que les valeurs numériques
df['price'] = df['price'].str.extract("([-+]?\d*\.\d+|[-+]?\d+)").astype(float)

In [ ]:
#Nous sauvegardons le mix et le max pour la normalisation
max_price = df['price'].max()
min_price = df['price'].min()

In [ ]:
#Nous effectuons une normalisation sur le prix pour avoir une valeur entre 0 et 1 , 0 voulant dire pas cher du tout et 1 trés cher
df['price_normalise'] = df['price'].apply(lambda x: (x - min_price) / (max_price - min_price))

JKDQFDSQHFQSJKL


In [ ]:
y = df['overall'].values

In [ ]:
type(model.sv.vectors)

In [ ]:
#Tout d'abord , nous recuillons les users de facon unique dans une liste
user_ids = df["reviewerID"].unique().tolist()
#Nous représentons chaque utilisateur de notre liste en un nombre entier et vice versa
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}

#Nous remplacons les colonnes de notre DF contenant les ID des utilisateurs et des articles sous format caractères par le format encodé en  nombre entier
df["reviewerID"] = df["reviewerID"].map(user2user_encoded)
#Nous récupérons la taille des vecteurs des utilisateurs et articles
num_users = len(user2user_encoded)


In [ ]:
df.columns

In [ ]:
df_x=df[['description_embedded','reviewerID','price_normalise']]

In [ ]:
df_x.dropna(inplace=True)

In [ ]:
x = df_x.values

In [ ]:
train_indices = int(0.7 * df.shape[0])

x_train, x_val, y_train, y_val = (
    x[:train_indices],
    x[train_indices:],
    y[:train_indices],
    y[train_indices:],
)


In [ ]:
model2 = Sequential()

model2.add(Dense(64,activation='relu'))

model2.add(Dense(32, activation='relu'))
model2.add(Dense(1,activation='linear'))
model2.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(lr=1e-2),metrics=[tf.keras.metrics.RootMeanSquaredError(),"mae"])
es = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)



In [ ]:
model2.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=keras.optimizers.Adam(lr=1e-2),metrics=[tf.keras.metrics.RootMeanSquaredError(),"mae"])

In [ ]:
history = model2.fit(
    x_train,
    y_train,
    epochs=200,
    validation_data=(x_val,y_val),
    callbacks=[es])

In [ ]:
df = reviews.merge(metadata,on='asin',)
df.columns

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'image_x', 'category', 'tech1', 'description', 'fit', 'title',
       'also_buy', 'image_y', 'tech2', 'brand', 'feature', 'rank', 'also_view',
       'main_cat', 'similar_item', 'date', 'price', 'details'],
      dtype='object')